# Predições privadas com Syft Keras

# Passo 1: Treinamento Público

Bem-vindo a este tutorial! Nos exemplos seguintes, você aprenderá como fornecer predições privadas. Por predições privadas, queremos dizer que os dados são constantemente encriptados ao longo de todo o processo. Em nenhum momento o usuário partilha os dados brutos, apenas dados encriptados (ou seja, compartilhamento secreto). A fim de fornecer essas predições privadas, a Syft Keras usa uma biblioteca chamada [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) por baixo dos panos. TF Encrypted combina criptografia de ponta (i.e cutting-edge cryptographic) e técnicas de aprendizagem de máquina, mas você não precisa se preocupar com isso e pode se concentrar na sua aplicação de aprendizagem de máquina.

Você pode começar a realizar predições privadas com apenas três passos:
- **Passo 1**: Treine seu modelo com o Keras normalmente.
- **Passo 2**: Proteja e disponibilize seu modelo de aprendizagem de máquina (server).
- **Passo 3**: Consulte o modelo seguro para receber predições privadas. 

Muito bem, vamos passar por estes três passos para que possa que você possa implementar serviços de aprendizagem de máquinas impactantes sem sacrificar a privacidade do usuário ou a segurança do modelo.

Autores:
- Jason Mancuso - Twitter: [@jvmancuso](https://twitter.com/jvmancuso)
- Yann Dupis - Twitter: [@YannDupis](https://twitter.com/YannDupis)
- Morten Dahl - Twitter: [@mortendahlcs](https://github.com/mortendahlcs)

Em nome de:
- Dropout Labs - Twitter: [@dropoutlabs](https://twitter.com/dropoutlabs)
- TF Encrypted - Twitter: [@tf_encrypted](https://twitter.com/tf_encrypted)

Tradução:
- Marcus Costa - Twitter: [@marcustpv](https://twitter.com/marcustpv)

## Treine seu modelo com o Keras

Para utilizar técnicas de privacidade em aprendizagem de máquina nos seus projetos, você não deve ter que aprender um novo _framework_ de aprendizagem de máquina. Se você tem conhecimentos básicos de [Keras](https://keras.io/), você pode começar a usar estas técnicas com Syft Keras. Se você nunca usou o Keras antes, você pode aprender um pouco mais sobre ele através da [documentação do Keras](https://keras.io). 

Antes de servir predições privadas, o primeiro passo é treinar o seu modelo com Keras como você faria normalmente. Como exemplo, vamos treinar um modelo para classificar dígitos escritos à mão. Para treinar este modelo vamos usar o conjunto de dados [MNIST](http://yann.lecun.com/exdb/mnist/).

Pegamos [este exemplo](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) emprestado do repositório de referência do Keras. Para treinar o seu modelo de classificação, basta executar a célula abaixo.

In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# dimensoes da imagem de entrada
img_rows, img_cols = 28, 28

# os dados, divididos em treino e teste
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convertendo os vetores de classes em matrizes de classses binarias.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Salve os pesos do seu modelo para futuras predições privadas

Ótimo! O seu modelo está treinado. Vamos salvar os pesos do modelo com a chamada `model.save()`. No próximo 
*notebook*, vamos carregar estes pesos no Syft Keras para começar a servir predições privadas. 

In [ ]:
model.save('short-conv-mnist.h5')